# PSTAT 234 Group Project: Hate Crime Data

__PSTAT 234 Group Project members: Wenlu Gou, Nhan Huynh, Zach Terner, and Laura Urbisci__  
__Presentation date: June 14th from 12:00 - 3:00 pm__

___

##Project overview:

For the PSTAT 234 Final Group Project, we analyzed the hate crimes data set found in the [FiveThirtyEight Github repositiory](https://github.com/fivethirtyeight/data/tree/master/hate-crimes). The data set contains reported hate crime statistics in the United States broken up by state as well as demographic information for five years before and ten days after Trump was elected President. Bash and Python were used for the analysis.  


The Jupyter notebook for this project is broken into a five sections: 
- Extracting the data
- Pre-analysis and visualization
- Analyzing the data
- Model building 
- Conclusion

The Jupyter notebook will be converted to slides using the steps outlined here (https://medium.com/@mjspeck/presenting-code-using-jupyter-notebook-slides-a8a3c3b59d67).

## Extracting the data 

This section of the notebook contains the packages needed to run all of the code in this notebook in addition to the code used for data extraction.  


We obtained data from the following sources:
1. The main data is based on a .csv file found in [FiveThirtyEight Github repositiory on hate crimes](https://github.com/fivethirtyeight/data/tree/master/hate-crimes)
2. We scraped a table on [Wikipedia](https://en.wikipedia.org/wiki/List_of_U.S._states_and_territories_by_religiosity) based on Pew Research Center's study to identify the overall religious status.
3. We scraped a table from an article published by the [Denver Post](https://www.denverpost.com/2015/10/08/chart-compare-the-average-age-in-each-u-s-state-2005-2014/) to obtain median age for each state. 
4. We also looked into how the country was divided during the Civil War to identify which states can be classfied as conservative  (**Note:** 2, 3, and 4 are the additional covariates we'd like add into the main data frame in 1). 

We first install  and import all the important packages for the analysis.
### Installation

In [ ]:
! pip install lxml

In [ ]:
# install plotly package
! pip install plotly

In [ ]:
! python -c "import plotly; plotly.tools.set_credentials_file(username='nhanhuynh', api_key='Y6hY7vdVGfrm6WJztBi3')"

In [ ]:
! pip install pydotplus

In [ ]:
! dot -Tps tree.dot -o tree.ps 

In [ ]:
!apt-get install libqtscript4-svg

In [ ]:
!pip install graphviz

In [ ]:
!apt-get install graphviz

### Import installed packages

In [ ]:
# importing all of the necessary python packages

import pandas as pd
import numpy as np
import scipy as sp
from sklearn.cluster import KMeans
from copy import deepcopy
from urllib.request import urlopen
from bs4 import BeautifulSoup
from lxml import html
import requests
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.plotly as py
import plotly.graph_objs as go
from numpy.random import randn
import collections
from sklearn.tree import DecisionTreeRegressor
from sklearn.decomposition import PCA
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso, LassoCV
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import scale 
import statsmodels.api as sm
from scipy import stats
import matplotlib
from IPython.display import Image  
from sklearn.tree import export_graphviz
import pydotplus
import graphviz as g

### Download data 

In [ ]:
%%bash
wget https://raw.githubusercontent.com/fivethirtyeight/data/master/hate-crimes/hate_crimes.csv

In [ ]:
! head hate_crimes.csv

In [ ]:
crimes = pd.read_csv("hate_crimes.csv")
crimes.info()

### Data scraping 



In [ ]:
res = requests.get("https://en.wikipedia.org/wiki/List_of_U.S._states_and_territories_by_religiosity")
soup = BeautifulSoup(res.content,"html5lib") # import content of the webpage in html 
table = soup.find_all('table')[1] # find the second table
df = pd.read_html(str(table))[0]
df = df.drop(df.index[[0,30,53,54,55,56]]).iloc[:,[0,4]] # delete records that are U.S. territories and select only first and last column
df.columns = ["state","percent_religious"]
df.percent_religious = df.percent_religious.str[-3:].str[0:2] # clean up the format
df.percent_religious = df. percent_religious.astype(float)
df.percent_religious = df. percent_religious/100
df.head()

In [ ]:
# check if all states in df are the same and in crimes:
stateDF = df.state.unique()
stateDF.sort()
stateCr = crimes.state.unique()
stateCr.sort()
stateDF == stateCr

In [ ]:
# merge df into crimes using state column:
crimes = pd.merge(crimes,df,on=["state"],how="inner")

In [ ]:
# List of states and their groups during the Civil War:
union = ["Maine","New York","New Hampshire","Vermont","Massachusetts","Connecticut","Rhode Island","Pennsylvania","New Jersey","Ohio", "Indiana", "Illinois", "Kansas", "Michigan", "Wisconsin", "Minnesota", "Iowa", "California", "Nevada", "Oregon"]
confederacy = ["Texas", "Arkansas", "Louisiana", "Tennessee", "Mississippi", "Alabama", "Georgia", "Florida", "South Carolina", "North Carolina", "Virginia"]


def civilWar(state):
  if state in union:
    return 'union'
  elif state in confederacy:
    return 'confe'
  else: return 'none'
  

# add the civilWar column:
crimes['civilWar'] = crimes['state'].apply(civilWar)
crimes.civilWar.unique()

In [ ]:
# convert civilWar column to categorical variable
crimes.civilWar = crimes.civilWar.astype('category')
crimes.civilWar.value_counts()

In [ ]:
age = requests.get("https://www.denverpost.com/2015/10/08/chart-compare-the-average-age-in-each-u-s-state-2005-2014/")
soup = BeautifulSoup(age.content, "lxml") 
table = soup.find_all('table')[0]
age = pd.read_html(str(table))[0].iloc[:,[0,2]]
age.columns =['state','medianAge']
age.head()

In [ ]:
# check if all states in age df are the same and in crimes:
stateAge = age.state.unique()
stateAge.sort()
stateAge == stateCr

In [ ]:
# merge Age df into Crimes df using state column:
crimes = pd.merge(crimes,age,on=["state"],how="inner")

In [ ]:
# Translate US states to two letter codes (in order to create US map in Plotly package)
postal = requests.get("https://www.infoplease.com/state-abbreviations-and-state-postal-codes")
soup = BeautifulSoup(postal.content, "lxml") 
table = soup.find_all('table')[0]
postal = pd.read_html(str(table))[0].iloc[:,[0,2]]
postal.columns = ["state","abbr"]
# merge postal df into crimes df using state column:
crimes = pd.merge(crimes,postal,on=["state"],how="inner")

In [ ]:
postal.head()

## Pre-analysis and visualization

### Data cleaning
In this study, we want to compare the hate crimes in two periods: (a) from 2010 - 2015 and (b) the first 10 days after Trump was elected President in the 58th quadrennial American presidential election. The statistics on hate crimes from these two periods are recorded in different scales as the unit in pre-election hate crimes is annually average (per 100,000 population) from 2010 to 2015 while post-election hate crimes is the 10-day average (per 100,000 population) from November 9 -18, 2016. We convert the pre-election group to the 10-day average unit before proceeding the analysis.  
<br>
In addition, there are three columns that have missing data. Observations are missing for the field that describes the share of the population that are not U.S. citizens in 2015 (`share_non_citizen`), hate crimes per 100,000 population from Nov. 9-18, 2016 (`hate_crimes_per_100k_splc`), and average annual hate crimes per 100,000 population in 2010-2015 (`avg_hatecrimes_per_100k_fbi`).

#### Fill missing values in the predictors

We observe that the column `share_non_citizen` has some missing values. We will impute these missing values based on an external source published on [Henry J Kaiser Family Foundation](https://www.kff.org/other/state-indicator/distribution-by-citizenship-status/?currentTimeframe=0&selectedRows=%7B%22states%22:%7B%22mississippi%22:%7B%7D%7D%7D&sortModel=%7B%22colId%22:%22Location%22,%22sort%22:%22asc%22%7D). Note: the time frame of the external table on population distribution by citizenship status in each state is in 2016 while `share_non_citizen` is recorded in 2015. According to the table, all three states (Maine, Mississippi, and South Dakota) with missing values have 1% of share of the population that are not U.S. citizens in 2016.  
<br>
We also compare the mean of share of the population that are not U.S. citizens between before and after filling the missing values , the difference is not significant. 




In [ ]:
crimes["fbi_standardized_pre"] = crimes["avg_hatecrimes_per_100k_fbi"]/36.525

crimes.info() # structure of data - we see there is missing data in share non citizen, hate crimes per 100K, ave hate crimes, and therefore fbi standardized

In [ ]:
crimes[['fbi_standardized_pre','hate_crimes_per_100k_splc']].describe()

We observe the presence of missing values in the data

In [ ]:
# number of missing values in pre-election group
crimes['fbi_standardized_pre'].isnull().sum()

In [ ]:
# number of missing values in post-election
crimes['hate_crimes_per_100k_splc'].isnull().sum()

In [ ]:
# create a data set that doesn't contain missing values in these groups and examine the density functions:
subCrimes = crimes[['fbi_standardized_pre','hate_crimes_per_100k_splc']].dropna() 

fig, ax = plt.subplots(figsize=(10,7))

sns.kdeplot(subCrimes['fbi_standardized_pre'],shade=True,label='Pre-election',color='b',bw=.1,alpha=0.5)
sns.kdeplot(subCrimes['hate_crimes_per_100k_splc'],shade=True,label='Post-election',color='r',bw=.1,alpha=.5)
ax.set(xlabel='Hate Crimes', ylabel='Density Function')

plt.show()

In [ ]:
trace0 = go.Box(x=subCrimes['fbi_standardized_pre'],name = 'Pre-election',boxpoints='all',jitter=0.5,pointpos=-5,boxmean=True)
trace1 = go.Box(x=subCrimes['hate_crimes_per_100k_splc'],name = 'Post-election',boxpoints='all',jitter=0.5,pointpos=-5,boxmean=True)

data = [trace0, trace1]
py.iplot(data)

#### Paired-test
We ran a paired t-test to see if the number of hate crimes before the election is different than after the election. We found using a level of significance of  $\alpha =5\%$ that there is a significant between the number of hate crimes when Trump was elected. 



In [ ]:
#Do the matched pairs t-test (can add it later)
sp.stats.ttest_rel(subCrimes['fbi_standardized_pre'],subCrimes['hate_crimes_per_100k_splc'])


#### Clustering - to fill in missing values in the predictors
We used clustering to impute the missing values in the response variables. 



In [ ]:
crimes = crimes.set_index('state') # set state column as row index
# run this code once

In [ ]:
crimes[crimes['share_non_citizen'].isnull()].share_non_citizen

In [ ]:
# mean of share_non_citizen before filling missing values
crimes.share_non_citizen.mean()

In [ ]:
# share_non_citizen has 3 missing values (NaN) in Main, Mississippi, and South Dakota. 
# Replace these missing values with 1% from the Kaiser Family Foundation:
crimes.loc[['Maine','Mississippi','South Dakota'],'share_non_citizen'] = .01
crimes.share_non_citizen.mean()

### Visualization
We use a variety of plots to visualize the data (i.e. pair plots, correlation plot, density plots
religous percentages by state plot, and boxplots). 


Transform the data from wide format to long format, then drop row IDs with missing values:

In [ ]:
colsToDrop = ['avg_hatecrimes_per_100k_fbi'] # remove some columns out 
crimesL = crimes.drop(colsToDrop, axis=1)
# renames some columns 
crimesL = crimesL.rename(columns={'hate_crimes_per_100k_splc': 'post', 'fbi_standardized_pre': 'pre'})

# convert crimes from wide format to long format:
crimesL = pd.melt(crimesL,id_vars=['median_household_income','share_unemployed_seasonal','share_population_in_metro_areas','share_population_with_high_school_degree','share_non_citizen','share_white_poverty', 'gini_index', 'share_non_white','share_voters_voted_trump','percent_religious', 'civilWar', 'medianAge','abbr'],var_name='time', value_name='hateCrimes')
crimesL = crimesL.dropna()

To understand the relationship between `share_voters_voted_trump` and `share_population_with_high_school_degree` on `hateCrimes` before and after the election, we can create a bubble plot as follows:

In [ ]:
post = crimesL[crimesL.time=="post"].loc[:,["share_voters_voted_trump","share_population_with_high_school_degree","hateCrimes","abbr"]]
post['abbr'] = post['abbr'].astype(str)
pre = crimesL[crimesL.time=="pre"].loc[:,["share_voters_voted_trump","share_population_with_high_school_degree","hateCrimes","abbr"]]
pre['abbr'] = pre['abbr'].astype(str)

fig, ax = plt.subplots(figsize=(15,12))

plt.scatter(np.array(pre.share_voters_voted_trump),y=np.array(pre.share_population_with_high_school_degree),s=np.array(pre.hateCrimes)*4000,data=pre,alpha=1,color="r")   
plt.scatter(x=np.array(post.share_voters_voted_trump),y=np.array(post.share_population_with_high_school_degree),s=np.array(post.hateCrimes)*4000,data=post,alpha=0.4,color="y")   

plt.axis([0,0.75,0.785,0.925])
ax.set_xlabel('Share voters voted for Trump')
ax.set_ylabel('Share population with high school degree')
ax.xaxis.label.set_size(12)
ax.yaxis.label.set_size(12)

z = randn(10)
yellow_dot, = plt.plot(z, "yo", markersize=15)
red_dot, =plt.plot(z, "ro", markersize=15)
plt.legend([yellow_dot, red_dot],["post-election","pre-election"],loc=2)

for i, txt in list(enumerate(np.array(pre.abbr))):
    plt.annotate(np.array(txt), (np.array(pre.share_voters_voted_trump)[i]+0.01,np.array(pre.share_population_with_high_school_degree)[i]), size=12, weight='bold',rotation=10)

plt.show()


From the above plot, we observe that DC seems to be an outlier as it is away from the bulk of the data points. Specifically, the distance between its value  and other states' values in `share_voters_voted_trump` is significant. The large difference in radius between after and post-election circles indicates hate crimes increases rapidly right after the election. Some states only have red circles as they have missing values in hate crimes after the election. We will handle the missing values in the next section. 

To better understand the relationship between these covariates on `hateCrimes` for other states except DC , let's "zoom in" the plot:

In [ ]:
# If remove DC and re-do the plot (to zoom in)
fig, ax = plt.subplots(figsize=(15,12))

plt.scatter(np.array(pre.share_voters_voted_trump),y=np.array(pre.share_population_with_high_school_degree),s=np.array(pre.hateCrimes)*7000,data=pre,alpha=1,color="r")   
plt.scatter(x=np.array(post.share_voters_voted_trump),y=np.array(post.share_population_with_high_school_degree),s=np.array(post.hateCrimes)*7000,data=post,alpha=.4,color="y")   
# add quantiles lines:
plt.axhline(y=crimesL.share_population_with_high_school_degree[crimesL['abbr']!="DC"].quantile(.5), color='r', linestyle=':')
plt.axvline(x=crimesL.share_voters_voted_trump[crimesL['abbr']!="DC"].quantile(.5), color='r', linestyle=':')

plt.axis([0.3,0.75,0.785,0.925])
ax.set_xlabel('Share voters voted for Trump')
ax.set_ylabel('Share population with high school degree')
ax.xaxis.label.set_size(12)
ax.yaxis.label.set_size(12)

z = randn(10)
yellow_dot, = plt.plot(z, "yo", markersize=15)
red_dot, =plt.plot(z, "ro", markersize=15)
plt.legend([yellow_dot, red_dot],["post-election","pre-election"],loc=2)

for i, txt in list(enumerate(np.array(pre.abbr))):
    plt.annotate(np.array(txt), (np.array(pre.share_voters_voted_trump)[i]+0.005,np.array(pre.share_population_with_high_school_degree)[i]), size=10, weight='bold',rotation=10)

plt.show()

This "zoom-in" plot provides some further insight about the relationship between these variables. We add two additional lines which are the medians of `share_voters_voted_trump` (the red vertical line) and `share_population_with_high_school_degree` (the red horizontal line). 
A lot of states in the upper left corner have large values in hate crimes compared to other states (the circles are bigger). The upper left corner is where the share voters voted for Trump below the national average and the share population with high school degree is above the national average. The rises in hate crimes after the election seems to be significant for these states.



We further compute the difference in hate crimes after and before the election (for only states without missing values for now), where:

$$diffCrimes = \text{hate_crimes_per_100k_splc} - \text{fbi_standardized_pre}$$

Then, we calculate the correlation matrix and visualize it:

---



In [ ]:
crimesCorr  = crimes.copy()
crimesCorr  = crimesCorr.dropna()
crimesCorr['diffCrimes'] = crimesCorr.hate_crimes_per_100k_splc - crimesCorr.fbi_standardized_pre

topdropCorr = ['civilWar','abbr','avg_hatecrimes_per_100k_fbi','hate_crimes_per_100k_splc','fbi_standardized_pre']
crimesCorr =  crimesCorr.drop(topdropCorr, axis=1)

In [ ]:
# Heat map for the correlation matrix
corr = crimesCorr.corr()

# Generate a mask for the upper triangle
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True
# Generate a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)

sns.set(style="white")

fig, ax = plt.subplots(figsize=(10, 8))
f1 = sns.heatmap(corr, mask=mask, cmap=cmap, xticklabels=corr.columns.values,yticklabels=corr.columns.values, vmax=.3, center=0,
            square=True, linewidths=.5)
plt.show()

In the correlation plot, observe that some covariates are highly correlated and their correlation makes good sense demographically. For example, `share_non_citizen` and `share_population_in_metro_areas` have a strong positive correlation (correlation coefficient is $.77$) or `share_white_poverty` and `median_household_income` have a negative correlation of $-.83$. We want to pay a close attention to these correlation for the model building procedures. 

Similarly, we can examine the relationships between the covariates on the difference in hate crimes. The bubble plot we created early reflects the negative correlation between `share_voters_voted_trump` and the rises in hate crimes. With the correlation matrix, we have a concrete coefficient to measure their linear relationship. If we assume there are no other variables in the data, the linear coefficient between the two variables is $-.64$. Indeed, for states that have lower `share_voters_voted_trump`, the difference in hate crimes tend to increase. 

We also notice that the additional variable we added into the data, `percent_religious`, has an intermediate and negative correlation with `diffCrimes`(coefficient of $-0.36$) while its correlation with `share_population_with_high_school_degree` is the highest ($-0.58$) among all the covariates available in the data. We can futher explore their relationship in the scatter plot below:

In [ ]:
fig, ax = plt.subplots(figsize=(15,7))

ax.grid(color='r', linestyle=':', linewidth=0.6)
plt.axhline(y=crimesCorr['diffCrimes'].quantile(.5), color='b', linestyle='-')

plt.scatter(x=crimesCorr['percent_religious'], y=crimesCorr['diffCrimes'], c=crimesCorr['share_population_with_high_school_degree'], 
            cmap='Spectral',s=250,edgecolors="k",linewidth=1.5)

ax.set_xlabel('Religious percentage')
ax.set_ylabel('Difference in crimes (Post-Pre)')
cbar = plt.colorbar()
cbar.set_label('Share population with high school degree')
plt.show()

We produce a scatter plot between the difference in hate crimes  (y-axis) versus the religious percentage (x-axis). We color each data point using the value of share population with high school degree. The blue line is the 50% quantile of the difference in hate crimes. Since `percent_religious` has a negative correlated coefficient with `diffCrimes`, we observe the linear and negative slope in the above plot. Based on the colorbar, states with the highest values in `share_population_with_high_school_degree` and small percentage in religion, difference in hate crimes seems to be higher than the average.  Similarly, states with the smallest values in `share_population_with_high_school_degree` and the greatest values in `percent_religious`, the difference in hate crimes is below the average level. 

We also create a pair plot to see the relationship between each pair of the covariates in the data:

In [ ]:
# create the scatter plot:
sns.set(style="ticks", color_codes=True)
g = sns.pairplot(crimesCorr,kind="reg",size=3,diag_kind="kde",diag_kws=dict(shade=True),palette="husl")

One of the additional variables in this data is `civilWar`, which is coded as a categorical variable. We can examine the distribution in difference in hate crimes for each level in `civilWar` to see if civil war factors affect the hate crimes:

In [ ]:
crimesCivil  = crimes.copy()
crimesCivil  = crimesCivil.dropna()
crimesCivil['diffCrimes'] = crimesCivil.hate_crimes_per_100k_splc - crimesCivil.fbi_standardized_pre

In [ ]:
trace0 = go.Box(x=crimesCivil.diffCrimes[crimesCivil['civilWar']=="union"],name = 'Union',boxpoints='all',jitter=0.5,pointpos=-5,boxmean=True)
trace1 = go.Box(x=crimesCivil.diffCrimes[crimesCivil['civilWar']=="confe"],name = 'Confederacy',boxpoints='all',jitter=0.5,pointpos=-5,boxmean=True)
trace2 = go.Box(x=crimesCivil.diffCrimes[crimesCivil['civilWar']=="none"],name = 'None',boxpoints='all',jitter=0.5,pointpos=-5,boxmean=True)

data = [trace0, trace1, trace2]
py.iplot(data)


K Means Clustering for the Hate Crime

In [ ]:
#Make a copy
crimes_deep_ref = deepcopy(crimes)

In [ ]:
crimes2 = crimes_deep_ref
type(crimes2)

#Look at the columns
crimes2.columns

covariates = crimes2.drop(columns=['avg_hatecrimes_per_100k_fbi','hate_crimes_per_100k_splc', 'fbi_standardized_pre','abbr','civilWar'], axis=1)


In [ ]:
print(covariates)

In [ ]:
kmeans = KMeans(n_clusters=6, random_state=0).fit(covariates)
kmeans.labels_

In [ ]:
#Add the cluster column as a column

crimes2["cluster_id"] = kmeans.labels_
#Impute using it by group_by cluster and finding the average for each one.

#Will require adding old columns also.

crimes2.head()

In [ ]:
crimes2[["abbr","cluster_id"]].sort_values(by=["cluster_id"])

#Clusters visualized below.

In [ ]:
#Find the frequency for each type of cluster
frq=collections.Counter(kmeans.labels_)
print(frq)

In [ ]:
#crimes2.info()

#Group by cluster ID to get the means of the hate crimes per 100k SPLC for the different clusters.
crimes2.groupby(["cluster_id"]).mean()["hate_crimes_per_100k_splc"]

In [ ]:
#Assign the means of each cluster to the corresponding missing entry.

#Find the states and cluster IDs with missing hate_crimes_per_100k_splc.
crimes2[crimes2["hate_crimes_per_100k_splc"].isnull()]["cluster_id"]



In [ ]:
#Assign them the missing values.
crimes2.loc["Hawaii","hate_crimes_per_100k_splc"] = .285794
crimes2.loc["North Dakota","hate_crimes_per_100k_splc"] = .356692
crimes2.loc["South Dakota","hate_crimes_per_100k_splc"] = .309873
crimes2.loc["Wyoming","hate_crimes_per_100k_splc"] = .309873


In [ ]:
#scl = [[0.0, 'rgb(242,240,247)'],[0.2, 'rgb(218,218,235)'],[0.4, 'rgb(188,189,220)'],[0.6, 'rgb(158,154,200)'],[0.8, 'rgb(117,107,177)'],[1.0, 'rgb(84,39,143)']]

data = [ dict(
        type='choropleth',
     
    #This colors it with rainbow colors.
        colorscale = 'Rainbow',
        autocolorscale = False,
        
    #This reverses the color scale
        #reversescale = True,
        locations = crimes['abbr'],
        z = crimes2['cluster_id'].astype(int),
        locationmode = 'USA-states',
        marker = dict(
             line = dict (
                color = 'rgb(255,255,255)',
                width = 2
            ) ),
        colorbar = dict(
            title = "Clustering based on covariates")
        )]

layout = dict(
        title = 'Clustering of United States by covariates',
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showlakes = True,
            lakecolor = 'rgb(255, 255, 255)'),
             )

fig = dict( data=data, layout=layout )
py.iplot( fig, filename='covariate-cluster-plot' )

##Model

1. found the difference of hate crimes before and after election
2. what else did we do? - add to this section




**Find the difference of hate crime before and after election**

In [ ]:
#Make a copy for Crimes2
crimes_diff = deepcopy(crimes2)
#Hawaii doesn't have fbi_standardized_pre data

In [ ]:
#Find the cluster0 fbi_standardized_pre mean
crimes_diff.groupby(["cluster_id"]).mean()["fbi_standardized_pre"][0]

In [ ]:
#Assign Hawaii the missing value by the group mean
crimes_diff.loc["Hawaii","fbi_standardized_pre"] = 0.065741

In [ ]:
#Find the difference between before and after election
crimes_diff["difference"] = crimes_diff["hate_crimes_per_100k_splc"] - crimes_diff["fbi_standardized_pre"]

In [ ]:
#Sort the difference 
crimes_diff[["abbr","difference"]].sort_values(by=["difference"])

In [ ]:
#Plot the differences 

#scl = [[0.0, 'rgb(242,240,247)'],[0.2, 'rgb(218,218,235)'],[0.4, 'rgb(188,189,220)'],[0.6, 'rgb(158,154,200)'],[0.8, 'rgb(117,107,177)'],[1.0, 'rgb(84,39,143)']]

data = [ dict(
        type='choropleth',
     
    #This colors it with rainbow colors.
        colorscale = 'Rainbow',
        autocolorscale = False,
        
    #This reverses the color scale
        #reversescale = True,
        locations = crimes['abbr'],
        z = crimes_diff['difference'].astype(float),
        locationmode = 'USA-states',
        marker = dict(
             line = dict (
                color = 'rgb(255,255,255)',
                width = 2
            ) ),
        colorbar = dict(
            title = "Scaled after-before difference in crimes")
        )]

layout = dict(
        title = 'Difference in hate crimes after the election',
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showlakes = True,
            lakecolor = 'rgb(255, 255, 255)'),
             )

fig = dict( data=data, layout=layout )
py.iplot( fig, filename='difference-crime-plot' )

###PCA

This section of code below does Principal Component Analysis or PCA.

In [ ]:
covariates.head()

In [ ]:
X_std = StandardScaler().fit_transform(covariates)

pca = PCA(n_components=2)

pca.fit(X_std)
print(pca.components_)

Interpretting PCA loadings:
- PCA1: (income, unemployed, metro, non citizen, gini, non white) vs (high school degreee, white povety, voted trump, percent religious, median age)
- PCA2: (income, high school degree, non citizen, median age) vs(unemployed, metro, white poverty, gini, non white, voted trump, percent religous)



In [ ]:
def myplot(score,coeff,labels=None):
    xs = score[:,0]
    ys = score[:,1]
    n = coeff.shape[0]
    scalex = 1.0/(xs.max() - xs.min())
    scaley = 1.0/(ys.max() - ys.min())
    plt.scatter(xs * scalex,ys * scaley)
    for i in range(n):
        plt.arrow(0, 0, coeff[i,0], coeff[i,1],color = 'r',alpha = 0.5)
        if labels is None:
            plt.text(coeff[i,0]* 1.15, coeff[i,1] * 1.15, "Var"+str(i+1), color = 'g', ha = 'center', va = 'center')
        else:
            plt.text(coeff[i,0]* 1.15, coeff[i,1] * 1.15, labels[i], color = 'g', ha = 'center', va = 'center')
plt.xlim(-1,1)
plt.ylim(-1,1)
plt.xlabel("PC{}".format(1))
plt.ylabel("PC{}".format(2))
plt.grid()

#Call the function. Use only the 2 PCs.
x_new = pca.fit_transform(X_std)

myplot(x_new,np.transpose(pca.components_))
#for i, txt in enumerate(covariates.index):
#    plt.annotate(txt, (x_new[:,0][i],x_new[:,1][i]))
plt.show()

In [ ]:
print(pca.explained_variance_)

In [ ]:
pca_all = PCA().fit(X_std)
objects = ('PCA1', 'PCA2', 'PCA3', 'PCA4', 'PCA5', 'PCA6', 'PCA7', 'PCA8', 'PCA9', 'PCA10', 'PCA11')
y_pos = np.arange(len(objects))

plt.bar(y_pos, pca_all.explained_variance_ratio_, align='center', alpha=0.75)
plt.plot(np.cumsum(pca_all.explained_variance_ratio_), color="orange", marker='o')
plt.xticks(y_pos, objects)
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance')
plt.title('Explained variance by different principal components')
plt.show()


###Regression Tree on difference

In [ ]:
#Define prediction indices
#predict_indices = [1,2,3,4,5,6,7,8,11,12,13,16,17]

#See if these work to get the columns we want
#crimes_diff.columns[predict_indices]


#Pick out the x and y variables for lasso usage
#Remove cluster_id but perhaps add it later as a dummy variable.
lasso_x = crimes_diff.drop(['hate_crimes_per_100k_splc','avg_hatecrimes_per_100k_fbi','civilWar','abbr','fbi_standardized_pre','cluster_id','difference'],axis = 1)

#Make crimes_diff a pandas dataframe
dummies_pd = pd.DataFrame(crimes_diff)

#Convert cluster_id to a string
dummies_pd['cluster_id'] = dummies_pd['cluster_id'].astype(str)

#Get the dummmies of cluster_ID and civilWar
dummies2 = pd.get_dummies(dummies_pd[['cluster_id','civilWar']])

#Concatenate the dummies to the lasso_x frame
lasso_x2 = pd.concat([pd.DataFrame(lasso_x), dummies2],axis=1)

#Make the response
lasso_y = crimes_diff['difference']

dummies2

In [ ]:
crimes_diff.columns

In [ ]:

alphas = 10**np.linspace(-1,-5,100)*0.5
#Fit Lasso to choose predictors?

lasso = Lasso(max_iter = 10000, normalize = True)
coefs = []

for a in alphas:
    lasso.set_params(alpha=a)
    lasso.fit(scale(lasso_x2), lasso_y)
    coefs.append(lasso.coef_)
    
ax = plt.gca()
ax.plot(alphas*2, coefs)
ax.set_xscale('log')
plt.axis('tight')
plt.xlabel('alpha')
plt.ylabel('weights')



In [ ]:
lassocv = LassoCV(alphas = None, cv = 10, max_iter = 100000, normalize = True)
lassocv.fit(lasso_x2, lasso_y)

lasso.set_params(alpha=lassocv.alpha_)
lasso.fit(lasso_x2, lasso_y)

pd.Series(lasso.coef_, index=lasso_x2.columns)

In [ ]:
# Create linear regression object
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(lasso_x2, lasso_y)

# Print coefficients with columns
pd.Series(regr.coef_, index=lasso_x2.columns)

X2 = sm.add_constant(lasso_x2)
est = sm.OLS(lasso_y, X2)
est2 = est.fit()
print(est2.summary())

In [ ]:
crimes_diff.head()

In [ ]:
tree_covariates = deepcopy(lasso_x2)

In [ ]:
# depth of tree: 5
reg_depth5 = DecisionTreeRegressor(max_depth=5)
reg_depth5_fit = reg_depth5.fit(tree_covariates,crimes_diff["difference"])
reg_depth5.feature_importances_

In [ ]:
importances1 = reg_depth5_fit.feature_importances_
plt.figure()
plt.title("Feature importances")
plt.bar(range(tree_covariates.shape[1]), importances1,color="r")
plt.xticks(range(tree_covariates.shape[1]))
plt.xlim([-1, tree_covariates.shape[1]])
plt.show()

When the depth of tree is 5. There are 8 features are not 0.

Those features are:median_household_income,

 share_unemployed_seasonal,

'share_population_in_metro_areas',

 'share_population_with_high_school_degree',
 
 'share_voters_voted_trump',
 
 'percent_religious',
 
 'medianAge', 'cluster_id_1'

The most important feature is share_voters_voted_trump

In [ ]:
reg_depth5_fit = reg_depth5.fit(tree_covariates,crimes_diff["difference"])
reg_depth5_fit

In [ ]:
# depth of tree: 4
#There are 6 features are significant different from 0
reg_depth4 = DecisionTreeRegressor(max_depth=4)
reg_depth4_fit = reg_depth4.fit(tree_covariates,crimes_diff["difference"])
reg_depth4.feature_importances_

In [ ]:
importances = reg_depth4.feature_importances_
plt.figure()
plt.title("Feature importances")
plt.bar(range(tree_covariates.shape[1]), importances,color="r")
plt.xticks(range(tree_covariates.shape[1]))
plt.xlim([-1, tree_covariates.shape[1]])
plt.show()

In [ ]:
#The list for features
list(tree_covariates)
#5 Importan features: median_household_income,'share_unemployed_seasonal','share_population_with_high_school_degree','share_voters_voted_trump','percent_religious'
#The most important feature is 'share_voters_voted_trump'

In [ ]:
%matplotlib inline

export_graphviz(reg_depth4,out_file='tree.dot')
#export_graphviz(reg_depth4)


In [ ]:
asdf = !cat tree.dot

In [ ]:
with open("tree.dot") as f:
    dot_graph = f.read()

# remove the display(...)

g.Source(dot_graph)

In [ ]:

Source.from_file('tree.dot', format='dot')

In [ ]:
dot = g.Digraph()

In [ ]:
df = lasso_x2
df['diff'] = lasso_y

In [ ]:

features

In [ ]:

## try to fit a regular linear model:
%%capture
#gather features
features = "+".join(lasso_x2.columns)

# get y and X dataframes based on this regression:
y, X = dmatrices('diff ~' + features, df, return_type='dataframe')


In [ ]:
from patsy import dmatrices
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm

In [ ]:
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif["features"] = X.columns

In [ ]:
vif.round(1)

# Conclusion

1. Also need to add to this section